In [1]:
%%latex
\tableofcontents

<IPython.core.display.Latex object>

# Ambulance_Dispatch_02_Binning_2024

## Goals of this Notebook
- Bin the values of each feature into up to some number of different codes, plus 99 for the values representing Missing or Unknown.
- We chose ten as the maximum number of codes for each feature (plus 99), but that's easy to change if you want to explore different options.

    

## Input
- CRSS_01.csv
    - There is no randomness involved in Ambulance_Dispatch_01_Get_Data
    - Merges the Accident, Vehicle, and Person datasets of CRSS
    - Undesirable features dropped
    - (817623, 67)


## Output
- CRSS_02_0_0.csv 
    - Random seed 0
    - Does not drop multilinear features
    - (802700, 67)
- CRSS_02_1_0.csv 
    - Random seed 1
    - Does not drop multilinear features
    - (802700, 67)
- CRSS_02_0_1.csv 
    - Random seed 0
    - Drops multilinear features
        - Drop  VE_FORMS
        - Drop  MAX_VSEV
        - Drop  VTCONT_F
    - (802700, 64)
- CRSS_02_1_1.csv 
    - Random seed 1
    - Drops multilinear features
        - Drop  VE_FORMS
        - Drop  MAX_VSEV
        - Drop  VTCONT_F
    - (802700, 64)
- Binning_Dict_0.json
     - Random seed 0
- Binning_Dict_1.json
    - Random seed 1
- The two binning dictionaries are the same, from comparison with ``diff``.
- The input dataset has 22,397 unique values in 67 features with a median 12 unique values per feature.
    - The outlier is DR_ZIP, the driver's zip code, with 
- The output datasets have 388 unique values in 67 features with a median 5 unique values per features if we do not reduce the dimensionality, and 375 unique values in 64 features with a median 5 if we do reduce the dimensionality.


## Number of Unique Values in Features Before and After Binning

- Note that most features include "99" as a value signifying "Missing or Unknown," so the medians of 12 and 5 really mean 11 and 4 known values and one that we will impute into those known values.

| Feature | Before | After |
|---|----|---|
| **Median** | 12 | 5 |
|  HOSPITAL  |  2  |  2  |
|  ACC_TYPE  |  93  |  11  |
|  AGE  |  119  |  11  |
|  AIR_BAG  |  11  |  6  |
|  ALC_STATUS  |  5  |  4  |
|  BODY_TYP  |  73  |  11  |
|  CARGO_BT  |  18  |  4  |
|  DAY_WEEK  |  7  |  5  |
|  DEFORMED  |  7  |  6  |
|  DR_ZIP  |  19297  |  11  |
|  EJECTION  |  7  |  4  |
|  HARM_EV  |  53  |  6  |
|  HIT_RUN  |  3  |  3  |
|  HOUR  |  25  |  8  |
|  IMPACT1  |  26  |  11  |
|  INJ_SEV  |  8  |  5  |
|  INT_HWY  |  3  |  3  |
|  J_KNIFE  |  4  |  3  |
|  LGT_COND  |  9  |  5  |
|  MAKE  |  71  |  11  |
|  MAK_MOD  |  1219  |  11  |
|  MAN_COLL  |  11  |  6  |
|  MAX_SEV  |  8  |  5  |
|  MAX_VSEV  |  8  |  5  |
|  MODEL  |  141  |  11  |
|  MONTH  |  12  |  6  |
|  M_HARM  |  53  |  6  |
|  NUMOCCS  |  65  |  6  |
|  NUM_INJ  |  19  |  7  |
|  NUM_INJV  |  17  |  5  |
|  PCRASH4  |  8  |  4  |
|  PCRASH5  |  9  |  4  |
|  PERMVIT  |  25  |  6  |
|  PER_TYP  |  3  |  3  |
|  PJ  |  425  |  10  |
|  PSU  |  60  |  10  |
|  PVH_INVL  |  12  |  3  |
|  P_CRASH1  |  20  |  8  |
|  P_CRASH2  |  57  |  11  |
|  REGION  |  4  |  4  |
|  RELJCT1  |  4  |  3  |
|  RELJCT2  |  15  |  5  |
|  REL_ROAD  |  13  |  4  |
|  REST_MIS  |  3  |  3  |
|  REST_USE  |  20  |  5  |
|  ROLINLOC  |  10  |  4  |
|  ROLLOVER  |  6  |  3  |
|  SEAT_POS  |  29  |  5  |
|  SEX  |  5  |  4  |
|  SPEC_USE  |  20  |  4  |
|  SPEEDREL  |  7  |  4  |
|  TOWED  |  7  |  6  |
|  TOW_VEH  |  9  |  4  |
|  TYP_INT  |  11  |  4  |
|  URBANICITY  |  2  |  2  |
|  VALIGN  |  7  |  5  |
|  VEH_AGE  |  88  |  11  |
|  VE_FORMS  |  14  |  4  |
|  VE_TOTAL  |  14  |  4  |
|  VPROFILE  |  9  |  6  |
|  VSPD_LIM  |  20  |  11  |
|  VSURCOND  |  13  |  4  |
|  VTCONT_F  |  7  |  4  |
|  VTRAFCON  |  19  |  4  |
|  VTRAFWAY  |  10  |  7  |
|  WEATHER  |  13  |  4  |
|  WRK_ZONE  |  5  |  3  |


## Background
- Each feature has at least two and up to several hundred values.
- Most features have one or more values signifying "Missing" or "Unknown."  
    - The previous notebook, Ambulance_Dispatch_01_Get_Data, creates a dictionary called Missing_Unknown_Dict that is saved in Big_Files as Missing_Unknown_Dict.json.
- All of the values are categorical.  
    - Some are ordered and linear, like age of person, number of lanes, speed limit,...
    - Some are ordered and periodic, particularly hour, day of week, and month.  We binned these by hand.
    - Most are unordered, like MAKE:  In which order would you put Buick, Renault, and Toyota?

## Method
- Drop all samples where HOSPITAL is missing or unknown.
- Bin HOSPITAL to [0,1] (meaning a list containing 0 and 1, not a closed interval)
For each feature:
- Make a dataframe with just that feature and the target variable, HOSPITAL.
- Delete all rows with missing or unknown values.
- Make a list of the unique (known) values of the feature.
- For each unique value, 
    - find the percentage of the set with that value, and
    - find the correlation between that value and the target variable, HOSPITAL, the proportion of the samples with that value for which HOSPITAL == 1.
- For unordered features, order the unique values by their correlation to HOSPITAL.
- As an example, MAKE has 68 unique values representing vehicle makes and three values for Missing/Unknown:
    - 97:  Not Reported
    - 98:  Other Make
    - 99:  Unknown Make
- This table shows, for each of the 68 MAKE codes, the percentage of crash persons (in a vehicle of known make) in a vehicle of that make, and the correlation (percentage of those persons who went) to the hospital.  


| Code | Per | Corr |
|---|---|---|
 |  [75]  |  0.0001  |  100.0  | 
 |  [71]  |  0.025  |  70.3518  | 
 |  [76]  |  0.3382  |  67.1625  | 
 |  [74]  |  0.0023  |  66.6667  | 
 |  [73]  |  0.2749  |  65.7365  | 
 |  [72]  |  0.9013  |  65.7318  | 
 |  [50]  |  0.0297  |  61.8644  | 
 |  [77]  |  0.0201  |  56.875  | 
 |  [53]  |  0.4319  |  49.5341  | 
 |  [64]  |  0.0015  |  33.3333  | 
 |  [43]  |  0.0005  |  25.0  | 
 |  [65]  |  0.0122  |  23.7113  | 
 |  [21]  |  0.1562  |  21.1755  | 
 |  [9]  |  0.0376  |  21.0702  | 
 |  [92]  |  0.0809  |  18.6625  | 
 |  [22]  |  0.8055  |  17.9547  | 
 |  [52]  |  0.8212  |  17.6417  | 
 |  [14]  |  0.4968  |  17.2152  | 
 |  [24]  |  0.4492  |  17.1613  | 
 |  [37]  |  9.5237  |  17.0835  | 
 |  [18]  |  1.4039  |  17.0116  | 
 |  [63]  |  3.3311  |  16.7894  | 
 |  [35]  |  8.247  |  16.4237  | 
 |  [36]  |  0.0635  |  16.0396  | 
 |  [55]  |  4.0438  |  15.8767  | 
 |  [6]  |  1.9401  |  15.6609  | 
 |  [69]  |  0.1693  |  15.3789  | 
 |  [20]  |  12.6771  |  15.1059  | 
 |  [13]  |  0.5906  |  14.9489  | 
 |  [34]  |  1.3292  |  14.8358  | 
 |  [39]  |  0.1026  |  14.8284  | 
 |  [47]  |  0.0438  |  14.6552  | 
 |  [67]  |  0.1239  |  14.5178  | 
 |  [49]  |  11.9024  |  14.3657  | 
 |  [25]  |  0.007  |  14.2857  | 
 |  [19]  |  0.9208  |  14.2721  | 
 |  [58]  |  0.9032  |  13.9933  | 
 |  [12]  |  13.3348  |  13.9543  | 
 |  [41]  |  1.488  |  13.9537  | 
 |  [30]  |  1.5739  |  13.7195  | 
 |  [2]  |  3.3535  |  13.6359  | 
 |  [42]  |  1.3751  |  13.5449  | 
 |  [7]  |  5.9545  |  13.2976  | 
 |  [54]  |  0.9884  |  13.1314  | 
 |  [23]  |  2.7177  |  12.7256  | 
 |  [59]  |  1.4425  |  12.6242  | 
 |  [93]  |  0.0101  |  12.5  | 
 |  [31]  |  0.0174  |  12.3188  | 
 |  [48]  |  1.3498  |  11.8979  | 
 |  [3]  |  0.055  |  11.4416  | 
 |  [62]  |  0.2026  |  10.6145  | 
 |  [38]  |  0.1742  |  10.3249  | 
 |  [45]  |  0.0917  |  10.1509  | 
 |  [32]  |  0.5354  |  10.007  | 
 |  [29]  |  0.1373  |  9.5238  | 
 |  [51]  |  0.6131  |  9.4154  | 
 |  [90]  |  0.0548  |  8.945  | 
 |  [10]  |  0.0015  |  8.3333  | 
 |  [94]  |  0.0218  |  5.7803  | 
 |  [84]  |  0.5081  |  4.9505  | 
 |  [89]  |  0.0136  |  4.6296  | 
 |  [86]  |  0.1836  |  4.5205  | 
 |  [82]  |  0.9525  |  4.1326  | 
 |  [85]  |  0.2953  |  4.0886  | 
 |  [87]  |  0.3452  |  3.4244  | 
 |  [1]  |  0.0004  |  0.0  | 
 |  [33]  |  0.0001  |  0.0  | 
 |  [46]  |  0.0001  |  0.0  | 


- We want to bin these 68 codes into a smaller number of bins.  For an example, we'll use 10 bins as our goal.
- First merge all of the codes with the same correlation to HOSPITAL, which is MAKE is only the codes with corr==0.0, meaning that of the crashes in that MAKE of vehicle, no one went to the hospital.  
- Now we have 66 bins.

| Code | Per | Corr |
|---|---|---|
 |  [75]  |  0.0001  |  100.0  | 
 |  [71]  |  0.025  |  70.3518  | 
 | ... | ... | ... |
 |  [82]  |  0.9525  |  4.1326  | 
 |  [85]  |  0.2953  |  4.0886  | 
 |  [87]  |  0.3452  |  3.4244  | 
 |  [1, 33, 46]  |  0.0006  |  0.0  | 


- Next find the smallest Per and merge it either into the bin above or below, whichever has the closest Corr.
    - Repeat until the smallest Per is 2%.  (This choice is arbitrary, and refining it is an opportunity for future research)
    - The smallest Per was 0.0001 with code [75], so merge it with [71].  
    - The new Per is the sum of the two Per's, and the Corr is the weighted average of the two.
    - The next smallest Per is 0.0005 with code [43], whose Corr of 25.0% is closer to the Corr of the bin below (23.7%) than that of the bin above (33.33%), so merge [43] with [65].

| Code | Per | Corr |
|---|---|---|
 |  [75, 71]  |  0.0252  |  70.5  | 
 |  [76]  |  0.3382  |  67.1625  | 
 |  [74]  |  0.0023  |  66.6667  | 
 |  [73]  |  0.2749  |  65.7365  | 
 |  [72]  |  0.9013  |  65.7318  | 
 |  [50]  |  0.0297  |  61.8644  | 
 |  [77]  |  0.0201  |  56.875  | 
 |  [53]  |  0.4319  |  49.5341  | 
 |  [64]  |  0.0015  |  33.3333  | 
 |  [43]  |  0.0005  |  25.0  | 
 |  [65]  |  0.0122  |  23.7113  | 
 |  [21]  |  0.1562  |  21.1755  | 
 
 | Code | Per | Corr |
|---|---|---|
 |  [75, 71]  |  0.0252  |  70.5  | 
 |  [76]  |  0.3382  |  67.1625  | 
 |  [74]  |  0.0023  |  66.6667  | 
 |  [73]  |  0.2749  |  65.7365  | 
 |  [72]  |  0.9013  |  65.7318  | 
 |  [50]  |  0.0297  |  61.8644  | 
 |  [77]  |  0.0201  |  56.875  | 
 |  [53]  |  0.4319  |  49.5341  | 
 |  [64]  |  0.0015  |  33.3333  | 
 |  [43, 65]  |  0.0127  |  23.7624  | 
 |  [21]  |  0.1562  |  21.1755  | 
 |  [9]  |  0.0376  |  21.0702  | 


- Continue until the smallest Per is at least 2.0%.
    - Now we're down to fifteen bins.

| Code | Per | Corr |
|---|---|---|
 |  [53, 75, 71, 74, 76, 73, 77, 50, 72]  |  2.0235  |  62.4301  | 
 |  [9, 64, 43, 65, 21, 92, 22, 52, 24, 14]  |  2.8616  |  17.8826  | 
 |  [18, 37]  |  10.9276  |  17.0743  | 
 |  [63]  |  3.3311  |  16.7894  | 
 |  [35]  |  8.247  |  16.4237  | 
 |  [6, 36, 55]  |  6.0474  |  15.8092  | 
 |  [69, 20]  |  12.8464  |  15.1094  | 
 |  [13, 39, 34]  |  2.0224  |  14.8685  | 
 |  [25, 19, 47, 67, 49]  |  12.9979  |  14.3615  | 
 |  [41, 58, 12]  |  15.7261  |  13.9565  | 
 |  [30, 42, 2]  |  6.3025  |  13.6369  | 
 |  [54, 7]  |  6.9428  |  13.2739  | 
 |  [23]  |  2.7177  |  12.7256  | 
 |  [29, 10, 90, 51, 38, 62, 45, 32, 3, 48, 31, 93, 59]  |  4.6853  |  11.3247  | 
 |  [94, 84, 1, 33, 46, 87, 85, 89, 86, 82]  |  2.3207  |  4.2486  | 

- Finally, find the smallest gaps in Corr and merge until we have at most ten bins.
    - Here, the smallest gap is between [69,20] at Corr of 15.1094 and [13,39,34] with Corr of 14.8685.  Merge those two.
- The choice of a maximum of ten bins was arbitrary, and searching for a better option is an opporunity for future research.

| Code | Per | Corr |
|---|---|---|
 |  [53, 75, 71, 74, 76, 73, 77, 50, 72]  |  2.0235  |  62.4301  | 
 |  [9, 64, 43, 65, 21, 92, 22, 52, 24, 14]  |  2.8616  |  17.8826  | 
 |  [18, 37]  |  10.9276  |  17.0743  | 
 |  [63]  |  3.3311  |  16.7894  | 
 |  [35]  |  8.247  |  16.4237  | 
 |  [6, 36, 55]  |  6.0474  |  15.8092  | 
 |  [69, 20, 13, 39, 34]  |  14.8688  |  15.0767  | 
 |  [25, 19, 47, 67, 49]  |  12.9979  |  14.3615  | 
 |  [41, 58, 12]  |  15.7261  |  13.9565  | 
 |  [30, 42, 2]  |  6.3025  |  13.6369  | 
 |  [54, 7]  |  6.9428  |  13.2739  | 
 |  [23]  |  2.7177  |  12.7256  | 
 |  [29, 10, 90, 51, 38, 62, 45, 32, 3, 48, 31, 93, 59]  |  4.6853  |  11.3247  | 
 |  [94, 84, 1, 33, 46, 87, 85, 89, 86, 82]  |  2.3207  |  4.2486  | 

- Here is the final binning with ten bins.  

| Code | Per | Corr |
|---|---|---|
 |  [50, 53, 71, 72, 73, 74, 75, 76, 77]  |  2.0235  |  62.4301  | 
 |  [9, 14, 21, 22, 24, 43, 52, 64, 65, 92]  |  2.8616  |  17.8826  | 
 |  [18, 37, 63]  |  14.2587  |  17.0077  | 
 |  [35]  |  8.247  |  16.4237  | 
 |  [6, 36, 55]  |  6.0474  |  15.8092  | 
 |  [13, 20, 34, 39, 69]  |  14.8688  |  15.0767  | 
 |  [2, 12, 19, 25, 30, 41, 42, 47, 49, 58, 67]  |  35.0265  |  14.0493  | 
 |  [7, 23, 54]  |  9.6606  |  13.1197  | 
 |  [3, 10, 29, 31, 32, 38, 45, 48, 51, 59, 62, 90, 93]  |  4.6853  |  11.3247  | 
 |  [1, 33, 46, 82, 84, 85, 86, 87, 89, 94]  |  2.3207  |  4.2486  | 

- Side Note:  Edge Cases.  We had to deal with two features, SPEC_USE and WRK_ZONE because the algorithm condensed them to just one bin. 
    - We earlier dropped features where one code was at least 99% of the dataset, but the most common value in SPEC_USE is only 98.84%.

| Code | Per | Corr |
|---|---|---|
 |  [19]  |  0.0038  |  48.3871  | 
 |  [1]  |  0.159  |  25.2147  | 
 |  [4]  |  0.0026  |  23.8095  | 
 |  [5]  |  0.2904  |  20.1709  | 
 |  [20]  |  0.0247  |  19.598  | 
 |  [10]  |  0.002  |  18.75  | 
 |  [8]  |  0.0073  |  16.9492  | 
 |  [3]  |  0.2186  |  15.7865  | 
 |  [0]  |  98.8412  |  15.6171  | 
 |  [21]  |  0.0262  |  14.6919  | 
 |  [6]  |  0.0989  |  10.2886  | 
 |  [22]  |  0.0122  |  9.1837  | 
 |  [13]  |  0.0014  |  9.0909  | 
 |  [2]  |  0.228  |  8.4377  | 
 |  [7]  |  0.0726  |  2.906  | 
 |  [23]  |  0.0081  |  1.5385  | 
 |  [11]  |  0.0007  |  0.0  | 
 |  [12]  |  0.0022  |  0.0  | 
 
-
    - If we follow our algorithm, iteratively finding the smallest PER and merging it into the nearest Corr, we get to:

| Code | Per | Corr |
|---|---|---|
 |  [19, 4, 1, 10, 20, 5]  |  0.4826  |  22.0422  | 
 |  [8, 3, 21, 0]  |  99.0933  |  15.6174  | 
 |  [6, 11, 12, 23, 7, 13, 22, 2]  |  0.4241  |  7.7553  | 
 
-
    - If we continued with the algorithm, merging until each bin accounted for at least 2% of the feature, then we would have reduced this feature to just one bin, which is silly.
    - Instead, we modified the algorithm to stop binning if the feature has three or fewer bins.


- Back to MAKE example:  Convert to a dictionary

{50: 0, 53: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 9: 1, 14: 1, 21: 1, 22: 1, 24: 1, 43: 1, 52: 1, 64: 1, 65: 1, 92: 1, 18: 2, 37: 2, 63: 2, 35: 3, 6: 4, 36: 4, 55: 4, 13: 5, 20: 5, 34: 5, 39: 5, 69: 5, 2: 6, 12: 6, 19: 6, 25: 6, 30: 6, 41: 6, 42: 6, 47: 6, 49: 6, 58: 6, 67: 6, 7: 7, 23: 7, 54: 7, 3: 8, 10: 8, 29: 8, 31: 8, 32: 8, 38: 8, 45: 8, 48: 8, 51: 8, 59: 8, 62: 8, 90: 8, 93: 8, 1: 9, 33: 9, 46: 9, 82: 9, 84: 9, 85: 9, 86: 9, 87: 9, 89: 9, 94: 9}

- Add the Missing/Unknown codes to the dictionary as bin 99

{50: 0, 53: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 9: 1, 14: 1, 21: 1, 22: 1, 24: 1, 43: 1, 52: 1, 64: 1, 65: 1, 92: 1, 18: 2, 37: 2, 63: 2, 35: 3, 6: 4, 36: 4, 55: 4, 13: 5, 20: 5, 34: 5, 39: 5, 69: 5, 2: 6, 12: 6, 19: 6, 25: 6, 30: 6, 41: 6, 42: 6, 47: 6, 49: 6, 58: 6, 67: 6, 7: 7, 23: 7, 54: 7, 3: 8, 10: 8, 29: 8, 31: 8, 32: 8, 38: 8, 45: 8, 48: 8, 51: 8, 59: 8, 62: 8, 90: 8, 93: 8, 1: 9, 33: 9, 46: 9, 82: 9, 84: 9, 85: 9, 86: 9, 87: 9, 89: 9, 94: 9, 97: 99, 98: 99, 99: 99}

- Append this dictionary to the master Binning_Dict

Binning_Dict = {

'ACC_TYPE': {6: 0, 50: 0, 51: 0, 52: 0, 53: 0, 55: 0, 58: 0, 59: 0, 60: 0, 61: 0, 1: 1, 4: 1, 10: 1, 14: 1, 2: 2, 5: 2, 7: 2, 16: 2, 54: 2, 0: 3, 3: 3, 8: 3, 9: 3, 41: 3, 62: 3, 64: 3, 69: 3, 89: 3, 66: 4, 83: 4, 87: 4, 90: 4, 91: 4, 30: 5, 34: 5, 65: 5, 68: 5, 82: 5, 86: 5, 88: 5, 11: 6, 12: 6, 22: 6, 24: 6, 25: 6, 26: 6, 31: 6, 32: 6, 35: 6, 38: 6, 39: 6, 42: 6, 73: 6, 77: 6, 79: 6, 85: 6, 15: 7, 21: 7, 27: 7, 29: 7, 33: 7, 40: 7, 43: 7, 48: 7, 71: 7, 72: 7, 75: 7, 80: 7, 81: 7, 13: 8, 20: 8, 23: 8, 28: 8, 44: 8, 45: 8, 47: 8, 49: 8, 67: 8, 74: 8, 76: 8, 78: 8, 84: 8, 36: 9, 37: 9, 46: 9, 56: 9, 57: 9, 63: 9, 70: 9, 92: 9, 93: 9, 98: 99, 99: 99}, 

'AGE': {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 3, 17: 3, 18: 4, 19: 5, 20: 5, 21: 6, 22: 6, 23: 6, 24: 6, 25: 6, 26: 6, 27: 6, 28: 6, 29: 6, 30: 6, 31: 6, 32: 6, 33: 6, 34: 6, 35: 6, 36: 6, 37: 6, 38: 6, 39: 6, 40: 6, 41: 6, 42: 6, 43: 6, 44: 6, 45: 6, 46: 6, 47: 6, 48: 6, 49: 6, 50: 7, 51: 7, 52: 7, 53: 7, 54: 7, 55: 7, 56: 7, 57: 7, 58: 7, 59: 7, 60: 7, 61: 7, 62: 7, 63: 7, 64: 8, 65: 8, 66: 8, 67: 8, 68: 8, 69: 8, 70: 8, 71: 8, 72: 9, 73: 9, 74: 9, 75: 9, 76: 9, 77: 9, 78: 9, 79: 9, 80: 9, 81: 9, 82: 9, 83: 9, 84: 9, 85: 9, 86: 9, 87: 9, 88: 9, 89: 9, 90: 9, 91: 9, 92: 9, 93: 9, 94: 9, 95: 9, 96: 9, 97: 9, 98: 9, 99: 9, 100: 9, 101: 9, 102: 9, 105: 9, 106: 9, 107: 9, 108: 9, 109: 9, 110: 9, 111: 9, 113: 9, 115: 9, 116: 9, 117: 9, 118: 9, 119: 9, 120: 9, 998: 99, 999: 99}, 

'AIR_BAG': {8: 0, 1: 1, 3: 2, 9: 2, 0: 3, 2: 3, 7: 3, 20: 4, 28: 4, 98: 99, 99: 99}, 

'ALC_STATUS': {1: 0, 2: 0, 0: 1, 8: 99, 9: 99},

...

 'MAKE': {50: 0, 53: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 9: 1, 14: 1, 21: 1, 22: 1, 24: 1, 43: 1, 52: 1, 64: 1, 65: 1, 92: 1, 18: 2, 37: 2, 63: 2, 35: 3, 6: 4, 36: 4, 55: 4, 13: 5, 20: 5, 34: 5, 39: 5, 69: 5, 2: 6, 12: 6, 19: 6, 25: 6, 30: 6, 41: 6, 42: 6, 47: 6, 49: 6, 58: 6, 67: 6, 7: 7, 23: 7, 54: 7, 3: 8, 10: 8, 29: 8, 31: 8, 32: 8, 38: 8, 45: 8, 48: 8, 51: 8, 59: 8, 62: 8, 90: 8, 93: 8, 1: 9, 33: 9, 46: 9, 82: 9, 84: 9, 85: 9, 86: 9, 87: 9, 89: 9, 94: 9, 97: 99, 98: 99, 99: 99}
 
...
 
 }

- Write the Binning_Dict to file "../../Big_Files/Binning_Dict.json"
    - Beware that in the original dictionary the keys are ints, but when you read in the .json dictionary, the keys are strings.  

- Use Pandas .replace() to bin the data
- Reduce the dimensionality
    - Drop features that are highly multicolinear with the other features
    - Keeping these features would just gum up the works in the next notebook where we impute missing data, and turn out to not be features we will not want to use in building our models for predicting needing an ambulance.  If they were, we could have made a list of features that can be dropped.
    - Remember that we are keeping many features that we will not use in the end to have more data for the imputation; then we will drop them.  We had thought VE_FORMS would be useful, but not if it's basically the same as VE_TOTAL. 
    - See Ambulance_Dispatch_2024_04_Reduce_Dimensionality for details on the method
    - This method drops five features
        - MAX_VSEV
        - VE_FORMS
        - VTCONT_F
        - MAX_SEV
        - NUM_INJV
- Save as '../../Big_Files/CRSS_Binned_Data.csv'
- Save a 10% sample and a n=1000 sample for future code development and debugging

# Setup
## Import Libraries

In [2]:
%matplotlib notebook

import sys, copy, math, time, statistics

print ('Python version: {}'.format(sys.version))

from IPython.display import display, HTML

from collections import Counter

import numpy as np
print ('NumPy version: {}'.format(np.__version__))
np.set_printoptions(suppress=True)

from numpy import array, linspace

import scipy as sc
print ('SciPy version: {}'.format(sc.__version__))
from scipy.signal import argrelextrema

import sklearn
print ('sklearn version: {}'.format(sklearn.__version__))
from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LinearRegression

import matplotlib
print ('Matplotlib version: {}'.format(matplotlib.__version__))
from matplotlib.pyplot import plot

import pandas as pd
print ('Pandas version:  {}'.format(pd.__version__))
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None 

import json # We will use json ('JavaScript Object Notation') to write and read dictionaries to/from files
print ('JSON version:  {}'.format(json.__version__))

# Set Randomness.  Copied from https://www.kaggle.com/code/abazdyrev/keras-nn-focal-loss-experiments
import random
random_seed = 1
print ('random_seed = ', random_seed)
np.random.seed(random_seed) # NumPy
random.seed(random_seed) # Python
#tf.set_random_seed(0) # Tensorflow


print ('Finished Importing Libraries')



Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:51:49) [Clang 16.0.6 ]
NumPy version: 1.26.4
SciPy version: 1.13.1
sklearn version: 1.5.0
Matplotlib version: 3.8.4
Pandas version:  2.2.2
JSON version:  2.0.9
random_seed =  1
Finished Importing Libraries


## Import Data
- Read the data file 
- Take out the NAME files and the IMputed files
- Read in the dictionary of feature values signifying "Missing" or "Unknown."

In [3]:
def Import_Stuff(file_number):
    print ('Import_Stuff()')
    filename_dict = {
        0: '../../Big_Files/CRSS_Merged_Raw_Data.csv',
        1: '../../Big_Files/CRSS_Merged_Raw_Data_Sample_frac_01.csv',
        2: '../../Big_Files/CRSS_Merged_Raw_Data_Sample_n_1000.csv',
        3: '../../Big_Files/CRSS_01.csv'
    }
    filename = filename_dict[file_number]
    print (filename)
    data = pd.read_csv(filename, index_col=None, low_memory=False)
    print ('data.shape with _IM features: ', data.shape)

    for feature in data:
        if 'NAME' in feature or '_IM' in feature:
            data.drop(columns=[feature], inplace=True)

    print ('data.shape without _IM features: ', data.shape)
    print ()
    
    print ('Reading in Missing/Unknown Dictionary')
    filename = '../../Big_Files/Missing_Unknown_Dict.json'
    with open(filename) as json_file:
        Missing_Unknown_Dict = json.load(json_file)
    print ()
    

    
    return data, Missing_Unknown_Dict

#Import_Data()


## List of Ordered Features
- Counts are ordered
- Some time features (YEAR, VEH_AGE) are ordered.
- Some time features (MONTH, HOUR, DAY_OF_WEEK) are ordered but periodic.
- They aren't continuous, so we still can't use SMOTE.

In [4]:
def Ordered_Unordered():
    Ordered = [
        'AGE',
        'ALC_RES',
        'DAY_WEEK',
        'HOUR',
        'MONTH',
        'NUMOCCS',
        'NUM_INJ',
        'NUM_INJV',
        'PVH_INVL',
        'VEH_AGE',
        'VE_FORMS',
        'VE_TOTAL',
        'VSPD_LIM',
    ]
    Unordered = [
        'ACC_TYPE',
        'AIR_BAG',
        'ALC_STATUS',
        'BODY_TYP',
        'CARGO_BT',
        'DEFORMED',
        'DR_ZIP',
        'HARM_EV',
        'HIT_RUN',
        'HOSPITAL',
        'IMPACT1',
        'INJ_SEV',
        'INT_HWY',
        'J_KNIFE',
        'LGT_COND',
        'MAKE',
        'MAK_MOD',
        'MAN_COLL',
        'MAX_SEV',
        'MAX_VSEV',
        'MODEL',
        'M_HARM',
        'PCRASH4',
        'PCRASH5',
        'PERMVIT',
        'PER_TYP',
        'PJ',
        'PSU',
        'P_CRASH1',
        'P_CRASH2',
        'REGION',
        'RELJCT1',
        'RELJCT2',
        'REL_ROAD',
        'REST_MIS',
        'REST_USE',
        'ROLINLOC',
        'ROLLOVER',
        'SEAT_POS',
        'SEX',
        'SPEC_USE',
        'SPEEDREL',
        'TOWED',
        'TOW_VEH',
        'TYP_INT',
        'URBANICITY',
        'VALIGN',
        'VPROFILE',
        'VSURCOND',
        'VTCONT_F',
        'VTRAFCON',
        'VTRAFWAY',
        'WEATHER',
        'WRK_ZONE',        
    ]
    
    return Ordered, Unordered

In [5]:
def Remove_Unknowns_in_Feature(data, Missing_Unknown_Dict, feature):
#    print ('Remove_Unknowns_in_Feature()')
#    print (feature)

    data.dropna(subset=[feature], inplace=True)

    if feature in Missing_Unknown_Dict.keys():
         data = data[~data[feature].isin(Missing_Unknown_Dict[feature])]
#        print (data.shape)
#        print (data[feature].unique())
#        print ()
#    print ()
    return data
    
    
    

## Bin 'HOUR'

In [6]:
def Bin_HOUR():

    a = """
# 0
[[0], 1.3659, 24.2163]
# 1
[[1], 1.0729, 27.6615]
[[2], 0.9663, 28.098]
[[3], 0.7437, 27.4662]
[[4], 0.7268, 26.2323]
# 2
[[5], 1.2066, 21.6472]
[[6], 2.4182, 17.4277]
# 3
[[7], 4.7249, 13.2663]
[[8], 4.5453, 13.5396]
# 4
[[9], 3.8083, 14.5844]
[[10], 4.0647, 15.1554]
[[11], 5.0718, 14.4391]
[[12], 6.2467, 13.7563]
[[13], 6.2725, 14.3218]
[[14], 7.0865, 14.4616]
# 5
[[15], 8.594, 13.1969]
[[16], 8.6698, 13.4154]
[[17], 9.2544, 12.9285]
[[18], 6.9822, 14.5054]
# 6
[[19], 4.7885, 16.7948]
[[20], 3.8226, 18.1821]
[[21], 3.2503, 19.1077]
[[22], 2.49, 20.6258]
# 0
[[23], 1.8271, 23.1265]
# 99 -> 99
"""
    HOUR_Dict = {
        0:0, 
        1:1, 2:1, 3:1, 4:1, 
        5:2, 6:2, 7:3, 8:3, 
        9:4, 10:4, 11:4, 12:4, 13:4, 14:4, 
        15:5, 16:5, 17:5, 18:5,
        19:6, 20:6, 21:6, 22:6, 
        23:0, 
        99:99
    }
    
    return HOUR_Dict


## Bin 'DAY_WEEK'

In [7]:
def Bin_DAY_WEEK():
    a = """
# 0
[[1], 11.3057, 18.6612]
# 1
[[2], 13.807, 15.0023]
# 2
[[3], 14.3335, 14.4461]
# 3
[[4], 14.6499, 14.7937]
[[5], 15.0209, 14.7084]
# 4
[[6], 17.0467, 14.3588]
# 0
[[7], 13.8363, 17.1742]
9 -> 99
"""
    DAY_WEEK_Dict = {
        1:0, 
        2:1, 
        3:2, 
        4:3, 5:3, 
        6:4, 
        7:0,
        9:99
    }
    
    return DAY_WEEK_Dict

## Bin 'MONTH'

In [8]:
def Bin_MONTH():
    a = """
# 0
[[1], 7.399, 14.5299]
[[2], 7.0764, 14.5805]
[[3], 7.7018, 15.04]
# 1
[[4], 7.3039, 15.9165]
[[5], 8.0828, 16.1499]
# 2
[[6], 8.3043, 16.8252]
[[7], 8.4636, 17.0246]
# 3
[[8], 9.0931, 16.2912]
[[9], 9.0098, 15.9137]
# 4
[[10], 9.8147, 14.9376]
[[11], 8.9343, 14.4027]
# 5
[[12], 8.8164, 13.7601]
    """
    MONTH_Dict = {
        1:0, 2:0, 3:0, 
        4:1, 5:1, 
        6:2, 7:2, 
        8:3, 9:3, 
        10:4, 11:4, 
        12:5
    }
    
    return MONTH_Dict

In [9]:
def Bin_Feature(data, A, C, U, feature, Print):
    if Print==1:
        print ('Bin_Feature()')
    
    # C has a row for each unique value of the feature.
    # C[0] is a list containing the value.
    # C[1] is the percentage of the dataset for that value, and 
    # C[2] is the correlation of that value to HOSPITAL.
    # The rows are sorted by decreasing D[1].
    
    if Print==1:
        print (feature)
        print (len(C))
#        display (C)
        
    if feature in ['SPEC_USE', 'WRK_ZONE']:
        print (len(C))
        for c in C:
            print (" | ", c[0], " | ", round(c[1],4), " | ", round(c[2],4), " | ")
        print ()

    # Merge equal values of C[1]
    for i in range (len(C)-1, 0, -1):
        if C[i-1][2] == C[i][2]:
            C[i-1][1] = C[i][1] + C[i-1][1]
            C[i-1][0] = C[i-1][0] + C[i][0]
            del(C[i])
            
#    if feature == 'MAKE':
#        print (len(C))
#        for c in C:
#            print (" | ", c[0], " | ", round(c[1],4), " | ", round(c[2],4), " | ")
#        print ()

#    display(C)
            
#    print ('Merge Small Per')
        
    # Merge values of C[1] less than 2% into the closest C[2]
    E = [c[1] for c in C]
    if len(E)>0:
        m = min(E)
        i = E.index(m)
    else:
        m = 1
#        data.drop(columns=[feature], inplace=True)
        print ('Drop ', feature, ' because it has devolved to just one value.')
    
    while m < 2.0 and len(C) > 3:        
        # Into which row are we going to merge row i ?
        if i==0:
            j=1
        elif i==len(C)-1:
            j=len(C)-2
        elif C[i][2] - C[i+1][2] > C[i-1][2] - C[i][2]: # We already took care of them being equal
            j = i-1
        else:
            j = i+1
        
#        print (i, C[i])
#        print (j, C[j])
        C[j][2] = (C[i][1]*C[i][2] + C[j][1]*C[j][2])/(C[i][1] + C[j][1])
        C[j][1] = C[i][1] + C[j][1]
        C[j][0] = C[i][0] + C[j][0]
#        print (C[j])
        del(C[i])
#        print (len(C))
#        print ()
        
        E = [c[1] for c in C]
        m = min(E)
        i = E.index(m)

        if feature in ['SPEC_USE', 'WRK_ZONE']:
            print (len(C))
            for c in C:
                print (" | ", c[0], " | ", round(c[1],4), " | ", round(c[2],4), " | ")
            print ()


#    if feature == 'MAKE':
#        print (len(C))
#        for c in C:
#            print (" | ", c[0], " | ", round(c[1],4), " | ", round(c[2],4), " | ")
#        print ()

#    print ()
#    print (len(C))
#    display(C)
#    print ('Merge Gaps')
    
    # Merge gaps less than 0.1%
    E = [abs(C[i][2] - C[i+1][2]) for i in range (0,len(C)-1)]
    if len(E) > 0:
        m = min(E)
        i = E.index(m)
    else:
        m = 1
        data.drop(columns=[feature], inplace=True)
        print ('Drop ', feature, ' because it has devolved into just one value.')
    
#    while m < 0.1:        
#    while len(C) > 10:        
    while m < 0.1 or len(C) > 10:
        j = i+1
#        print (i, C[i])
#        print (j, C[j])
        C[j][2] = (C[i][1]*C[i][2] + C[j][1]*C[j][2])/(C[i][1] + C[j][1])
        C[j][1] = C[i][1] + C[j][1]
        C[j][0] = C[i][0] + C[j][0]
#        print (C[j])
        del(C[i])
#        print (len(C))
#        print ()
        
        E = [abs(C[i][2] - C[i+1][2]) for i in range (0,len(C)-1)]
        m = min(E)
        i = E.index(m)

#        if feature == 'MAKE':
#            print (len(C))
#            for c in C:
#                print (" | ", c[0], " | ", round(c[1],4), " | ", round(c[2],4), " | ")
#            print ()


#    print ()
#    print (len(C))
#    display(C)

    for c in C:
        c[0].sort()
        
#    if feature == 'MAKE':
#        print (len(C))
#        for c in C:
#            print (" | ", c[0], " | ", round(c[1],4), " | ", round(c[2],4), " | ")
#        print ()

        
    for i in range (len(C)-1):
        C[i].append(abs(C[i][2] - C[i+1][2]))
        C[i].append(i)
    
#    print (len(C))
#    display ([[c[1],c[2], len(c[0]), sorted(c[0])] for c in C])
        
    Feature_Binning_Dict = {}
    for i in range (len(C)):
        for c in C[i][0]:
            Feature_Binning_Dict[int(c)] = int(i)
            
#    if feature=='MAKE':
#        print (Feature_Binning_Dict)
            
        
    return Feature_Binning_Dict
    

In [10]:
def Binning(data, target, Missing_Unknown_Dict, Ordered, Unordered, Print):
    
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html
    print ("Binning")
    
    Binning_Dict = {}
    
    for feature in data:
#    for feature in ['ACC_TYPE']:
        print (feature, len(data[feature].unique()))
#        display (data[feature].value_counts())
        if feature != target:
            A = Remove_Unknowns_in_Feature(data[[target, feature]], Missing_Unknown_Dict, feature)
            U = sorted(A[feature].unique())
            C = []
            for u in U:
                B = A[A[feature]==u]
                TP = B[target].sum()
                PP = len(B)
                corr = TP/PP*100
                per = len(B)/len(A)*100

                C.append([[u], per, corr])

            if feature in Unordered:
                if Print==1:
                    print ('Sort Unordered Feature')
                C.sort(key=lambda x:x[2], reverse=True)

            if feature == 'SPEC_USE':
                print (feature)
                print (len(C))
                for c in C:
                    print (" | ", c[0], " | ", round(c[1],4), " | ", round(c[2],4), " | ")
                print ()

            """
            data[feature] is a Pandas series of the values of the feature for each the 800,000+ samples
            A is the a Pandas dataframe with two columns:
                    HOSPITAL (the target feature)
                    feature (the current feature)
                with the samples that have "Missing" and "Unknown" values for the current feature removed
            U is a list of the unique values in A
            for each unique value u in U:
                B is the Pandas series A filtered to just have the samples with that unique value u
            C is a list, for each unique value u in U, of:
                u, the unique value, 
                per, the percentage of the samples in list A that has that value
                corr (correlation), the percent of the samples that have this value that are hospitalized

            """

            Feature_Binning_Dict = Bin_Feature(data, A, C, U, feature, Print)
                
            if Print==1:
                print (feature, len(U))
#                print (Feature_Binning_Dict)
                print (Missing_Unknown_Dict[feature])
#                print ()

            # Put the missing and unknown values back in as 99.
            for mu in Missing_Unknown_Dict[feature]:
                Feature_Binning_Dict[mu] = 99

            if feature == 'HOUR':
                Feature_Binning_Dict = Bin_HOUR()
            if feature == 'DAY_WEEK':
                Feature_Binning_Dict = Bin_DAY_WEEK()
            if feature == 'MONTH':
                Feature_Binning_Dict = Bin_MONTH()

            Binning_Dict[feature] = Feature_Binning_Dict

            if Print==1 or feature == 'SPEC_USE':
                print (list(set(list(Feature_Binning_Dict.values()))))
                print ()
                print(Feature_Binning_Dict)
                    

            
    return Binning_Dict

In [11]:
def Make_List_of_Features(data, Missing_Unknown_Dict, target):
    print ('Features and number of unique values in feature')
    
    B = []
    for feature in data:
        if feature != target:
            A = Remove_Unknowns_in_Feature(data[[target, feature]], Missing_Unknown_Dict, feature)
            U = sorted(A[feature].unique())
            B.append([feature, len(U)])
    B.sort(key = lambda x:x[0])
    B.sort(key = lambda x:x[1])
    for b in B:
        print ('    %s, # %d' % (b[0], b[1]))
    print ()

## Reduce Dimensionality by Multicolinearlity

In [12]:
# Adapted from https://towardsdatascience.com/statistics-in-python-collinearity-and-multicollinearity-4cc4dcd82b3f
def calculate_vif(df, features):    
    r2_Dict, tolerance, vif = {}, {}, {}
    # all the features that you want to examine
    for feature in features:
        # extract all the other features you will regress against
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]
        # extract r-squared from the fit
        r2 = LinearRegression().fit(X, y).score(X, y)
        r2_Dict[feature] = r2
        # calculate tolerance
        tolerance[feature] = 1 - r2
        # calculate VIF
        if tolerance[feature] !=0:
            vif[feature] = 1/(tolerance[feature])
        else:
            vif[feature] = 10000

    return pd.DataFrame({'r2': r2_Dict, 'Tolerance': tolerance, 'VIF': vif}), tolerance, r2_Dict

# Iteratively remove the feature with the largest VIF ('Variance Inflaction Factor')
# until the largest VIF is 10, or smallest Tolerance is 0.1, or largest R^2 is 0.9
def Reduce_Dimensionality(data, target):
    print ()
    print ('Reduce_Dimensionality()')
    print ('data.shape: ', data.shape)
    Target = data.pop(target)
    Features = [feature for feature in data]
    VIF, Tolerance_Dict, r2_Dict = calculate_vif(data, Features)
    Max_r2_Feature = VIF['r2'].idxmax()
    display(VIF)
    print (Max_r2_Feature)
    if r2_Dict[Max_r2_Feature] > 0.9:
        data.drop(columns = [Max_r2_Feature], inplace=True)
        print ('Drop ', Max_r2_Feature)
    print ()
    while r2_Dict[Max_r2_Feature] > 0.9:
        Features = [feature for feature in data]
        VIF, Tolerance_Dict, r2_Dict = calculate_vif(data, Features)
        Max_r2_Feature = VIF['r2'].idxmax()
        display(VIF)
        print (Max_r2_Feature)
        if r2_Dict[Max_r2_Feature] > 0.9:
            data.drop(columns = [Max_r2_Feature], inplace=True)
            print ('Drop ', Max_r2_Feature)
        print ()

    data = data.join(Target)
    print ('data.shape: ', data.shape)
    print ()
        
    return data
        
    

In [13]:
def Count_Categories(data):
    print ('Count_Categories')
    n = 0
    N = []
    A = []
    for feature in data:
        s = len(data[feature].unique())
        n = n + s
        N.append(s)
        A.append([feature, s])
        
    print ('Total number of unique values =', n)
    print ('Median number of unique values =', statistics.median(N))
    print ()
    return n, A

In [14]:
%%time
def Main():
    target = 'HOSPITAL'
    data, Missing_Unknown_Dict = Import_Stuff(3)
    # Alphabetize the features
    data = data.reindex(sorted(data.columns), axis=1)
    # Put HOSPITAL first
    data = data[ [target] + [col for col in data.columns if col != target]]
    
    nUnique_Values, Feature_nUnique_Values = Count_Categories(data)
    
    # Get the lists of ordered and unordered sets
    Ordered, Unordered = Ordered_Unordered()
    
#    print ('Missing_Unknown_Dict')
#    print (Missing_Unknown_Dict)
#    print ()
    
#    Make_List_of_Features(data, Missing_Unknown_Dict, target)
    
    # Make binning dictionary
    Print = 0 # Change to Print=1 for Verbose
    Binning_Dict = Binning(data, target, Missing_Unknown_Dict, Ordered, Unordered, Print)

#    for key in Binning_Dict.keys():
#        print (key)
#        for item in Binning_Dict[key]:
#            print (item, type(item))
#    print ()

    # Write the binning dictionary as a json file

    if random_seed==0:
        filename = 'Binning_Dict_0.json'
    else:
        filename = 'Binning_Dict_1.json'
    with open("../../Big_Files/" + filename, "w") as outfile: 
        json.dump(Binning_Dict, outfile)
    outfile.close()

#    print ('Reading in Binning Dict')
#    with open('../../Big_Files/Binning_Dict.json') as json_file:
#        Binning_Dict = json.load(json_file)
#    json_file.close()
    
    # Bin the features according to Binning_Dict
    for feature in data:
        print (feature)
        if feature in Binning_Dict.keys():
            nU = len(data[feature].unique())
            print (nU, ' unique features in original data')
            data[feature].replace(Binning_Dict[feature], inplace=True)
            U = sorted(list(data[feature].unique()))
            V = list(set(Binning_Dict[feature].values()))
            print (U, ' values in binned feature')
            print (V, ' values in dictionary')
#            print (Binning_Dict[feature])
        else:
            print (feature, ' not in Binning_Dict')
        print ()
        
    for feature in data:
        data[feature] = pd.to_numeric(data[feature])
    data = data.astype('int64')
    
    print ('Drop rows with more than 20% of features missing')
    print (data.shape)
    data['new'] = data.isin({99}).sum(1)
    data.drop(data[data['new'] > 0.2 * data.shape[1]].index, inplace=True)
    data.drop(columns=['new'], inplace=True)
    print (data.shape)
    

    
    print ('data.shape: ', data.shape)
    
    # Write to file
    
#    data.to_csv('../../Big_Files/CRSS_02_Binned_Data.csv', index=False)
    # The first '_0' or '_1' is for random seed 0 or 1
    # The second '_0' is for not reducing the dimensionality before the next step, imputing.
    if random_seed==0:
        data.to_csv('../../Big_Files/CRSS_02_0_0.csv', index=False)
    else:
        data.to_csv('../../Big_Files/CRSS_02_1_0.csv', index=False)

    nUnique_Values, Feature_nUnique_Values_2 = Count_Categories(data)
    print ('| Feature | Unique Values Before Binning | Unique Values After Binning |')
    print ('|---|----|---|')
    for i in range(len(Feature_nUnique_Values)):
        print (
            '| ', Feature_nUnique_Values[i][0], 
            ' | ', Feature_nUnique_Values[i][1], 
            ' | ', Feature_nUnique_Values_2[i][1], 
            ' |'
        )
    print ()

    data = Reduce_Dimensionality(data, target)
    print ('data.shape: ', data.shape)
        
#    data.to_csv('../../Big_Files/CRSS_02_Binned_Reduced_Dimensionality_Data.csv', index=False)
    # The first '_0' or '_1' is for random seed 0 or 1
    # The second '_1' is for reducing the dimensionality before the next step, imputing.
    if random_seed==0:
        data.to_csv('../../Big_Files/CRSS_02_0_1.csv', index=False)
    else:
        data.to_csv('../../Big_Files/CRSS_02_1_1.csv', index=False)
        
    nUnique_Values, Feature_nUnique_Values_3 = Count_Categories(data)
    for row in Feature_nUnique_Values_3:
        print (row)
    print ()
        
    
    
    print ('Finished Binning Data')
        
Main()

#CPU times: user 29min 17s, sys: 1min 41s, total: 30min 59s
#Wall time: 10min 22s

Import_Stuff()
../../Big_Files/CRSS_01.csv
data.shape with _IM features:  (817623, 86)
data.shape without _IM features:  (817623, 67)

Reading in Missing/Unknown Dictionary

Count_Categories
Total number of unique values = 22397
Median number of unique values = 12

Binning
HOSPITAL 2
ACC_TYPE 93
AGE 119
AIR_BAG 11
ALC_STATUS 5
BODY_TYP 73
CARGO_BT 18
DAY_WEEK 7
DEFORMED 7
DR_ZIP 19297
EJECTION 7
HARM_EV 53
HIT_RUN 3
HOUR 25
IMPACT1 26
INJ_SEV 8
INT_HWY 3
J_KNIFE 4
LGT_COND 9
MAKE 71
MAK_MOD 1219
MAN_COLL 11
MAX_SEV 8
MAX_VSEV 8
MODEL 141
MONTH 12
M_HARM 53
NUMOCCS 65
NUM_INJ 19
NUM_INJV 17
PCRASH4 8
PCRASH5 9
PERMVIT 25
PER_TYP 3
PJ 425
PSU 60
PVH_INVL 12
P_CRASH1 20
P_CRASH2 57
REGION 4
RELJCT1 4
RELJCT2 15
REL_ROAD 13
REST_MIS 3
REST_USE 20
ROLINLOC 10
ROLLOVER 6
SEAT_POS 29
SEX 5
SPEC_USE 20
SPEC_USE
18
 |  [19]  |  0.0038  |  48.3871  | 
 |  [1]  |  0.159  |  25.2147  | 
 |  [4]  |  0.0026  |  23.8095  | 
 |  [5]  |  0.2904  |  20.1709  | 
 |  [20]  |  0.0247  |  19.598  | 
 |  [10

<timed exec>:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 99]  values in binned feature
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 99]  values in dictionary

AGE
119  unique features in original data
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 99]  values in binned feature
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 99]  values in dictionary

AIR_BAG
11  unique features in original data
[0, 1, 2, 3, 4, 99]  values in binned feature
[0, 1, 2, 3, 4, 99]  values in dictionary

ALC_STATUS
5  unique features in original data
[0, 1, 2, 99]  values in binned feature
[0, 1, 2, 99]  values in dictionary

BODY_TYP
73  unique features in original data
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 99]  values in binned feature
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 99]  values in dictionary

CARGO_BT
18  unique features in original data
[0, 1, 2, 99]  values in binned feature
[0, 1, 2, 99]  values in dictionary

DAY_WEEK
7  unique features in original data
[0, 1, 2, 3, 4]  values in binned feature
[0, 1, 2, 3, 4, 99]  values in dictionary

DEFORMED
7  unique features in original data

Drop rows with more than 20% of features missing
(817623, 67)
(802700, 67)
data.shape:  (802700, 67)
Count_Categories
Total number of unique values = 388
Median number of unique values = 5

| Feature | Unique Values Before Binning | Unique Values After Binning |
|---|----|---|
|  HOSPITAL  |  2  |  2  |
|  ACC_TYPE  |  93  |  11  |
|  AGE  |  119  |  11  |
|  AIR_BAG  |  11  |  6  |
|  ALC_STATUS  |  5  |  4  |
|  BODY_TYP  |  73  |  11  |
|  CARGO_BT  |  18  |  4  |
|  DAY_WEEK  |  7  |  5  |
|  DEFORMED  |  7  |  6  |
|  DR_ZIP  |  19297  |  11  |
|  EJECTION  |  7  |  4  |
|  HARM_EV  |  53  |  6  |
|  HIT_RUN  |  3  |  3  |
|  HOUR  |  25  |  8  |
|  IMPACT1  |  26  |  11  |
|  INJ_SEV  |  8  |  5  |
|  INT_HWY  |  3  |  3  |
|  J_KNIFE  |  4  |  3  |
|  LGT_COND  |  9  |  5  |
|  MAKE  |  71  |  11  |
|  MAK_MOD  |  1219  |  11  |
|  MAN_COLL  |  11  |  6  |
|  MAX_SEV  |  8  |  5  |
|  MAX_VSEV  |  8  |  5  |
|  MODEL  |  141  |  11  |
|  MONTH  |  12  |  6  |
|  M_HARM  |  53  |

,r2,Tolerance,VIF
ACC_TYPE,0.224320,0.775680,1.289192
AGE,0.495453,0.504547,1.981975
AIR_BAG,0.314344,0.685656,1.458456
ALC_STATUS,0.156342,0.843658,1.185315
BODY_TYP,0.824778,0.175222,5.707055
CARGO_BT,0.154171,0.845829,1.182272
DAY_WEEK,0.011758,0.988242,1.011898
DEFORMED,0.248485,0.751515,1.330645
DR_ZIP,0.322594,0.677406,1.476220
EJECTION,0.275861,0.724139,1.380950


...

MAX_VSEV
Drop  MAX_VSEV



,r2,Tolerance,VIF
ACC_TYPE,0.220704,0.779296,1.283210
AGE,0.495436,0.504564,1.981909
AIR_BAG,0.314303,0.685697,1.458371
ALC_STATUS,0.156071,0.843929,1.184934
BODY_TYP,0.824778,0.175222,5.707049
CARGO_BT,0.153736,0.846264,1.181664
DAY_WEEK,0.011720,0.988280,1.011859
DEFORMED,0.248409,0.751591,1.330512
DR_ZIP,0.322587,0.677413,1.476205
EJECTION,0.272831,0.727169,1.375195


VTCONT_F
Drop  VTCONT_F



,r2,Tolerance,VIF
ACC_TYPE,0.220704,0.779296,1.283210
AGE,0.495435,0.504565,1.981905
AIR_BAG,0.314272,0.685728,1.458304
ALC_STATUS,0.156027,0.843973,1.184872
BODY_TYP,0.824778,0.175222,5.707047
CARGO_BT,0.153735,0.846265,1.181663
DAY_WEEK,0.011720,0.988280,1.011859
DEFORMED,0.248359,0.751641,1.330422
DR_ZIP,0.322578,0.677422,1.476185
EJECTION,0.272756,0.727244,1.375053


NUM_INJ

data.shape:  (802700, 64)

data.shape:  (802700, 64)
Count_Categories
Total number of unique values = 375
Median number of unique values = 5.0

['ACC_TYPE', 11]
['AGE', 11]
['AIR_BAG', 6]
['ALC_STATUS', 4]
['BODY_TYP', 11]
['CARGO_BT', 4]
['DAY_WEEK', 5]
['DEFORMED', 6]
['DR_ZIP', 11]
['EJECTION', 4]
['HARM_EV', 6]
['HIT_RUN', 3]
['HOUR', 8]
['IMPACT1', 11]
['INJ_SEV', 5]
['INT_HWY', 3]
['J_KNIFE', 3]
['LGT_COND', 5]
['MAKE', 11]
['MAK_MOD', 11]
['MAN_COLL', 6]
['MAX_SEV', 5]
['MODEL', 11]
['MONTH', 6]
['M_HARM', 6]
['NUMOCCS', 6]
['NUM_INJ', 7]
['NUM_INJV', 5]
['PCRASH4', 4]
['PCRASH5', 4]
['PERMVIT', 6]
['PER_TYP', 3]
['PJ', 10]
['PSU', 10]
['PVH_INVL', 3]
['P_CRASH1', 8]
['P_CRASH2', 11]
['REGION', 4]
['RELJCT1', 3]
['RELJCT2', 5]
['REL_ROAD', 4]
['REST_MIS', 3]
['REST_USE', 5]
['ROLINLOC', 4]
['ROLLOVER', 3]
['SEAT_POS', 5]
['SEX', 4]
['SPEC_USE', 4]
['SPEEDREL', 4]
['TOWED', 6]
['TOW_VEH', 4]
['TYP_INT', 4]
['URBANICITY', 2]
['VALIGN', 5]
['VEH_AGE', 11]
['VE_TOTAL', 4]
[